In [36]:
import json
from http import HTTPStatus
import dashscope
from dotenv import dotenv_values
from retry import retry
config = dotenv_values('.env')

In [37]:
MODEL_NAME = 'qwen-max'
dashscope.api_key = config['qwen_key'],

@retry(delay=60, tries=3)
def call_qwen_api(MODEL_NAME, query):
    messages = [
        {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        #print(response)
        return response['output']['choices'][0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

In [38]:
def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""

    return prompt

In [39]:
def get_question_prompt(data):
    prompt = f"""# Role 
你是一个逻辑推理问题出题专家。以下是一个包含问题的 JSON 数据。请在原有子问题的基础上，额外生成三个新的子问题，保持格式一致并给出参考答案。

## Skills
1. 逻辑推理问题的设计与编写
2. 生成符合既定格式的题目
3. 提供高质量的参考答案

## Rules
1. 新生成的子问题需与原有问题逻辑一致。
2. 保持 JSON 数据的格式统一。
3. 提供的参考答案应清晰、准确。
4. 仅按照JSON格式回复。

## Workflows
1. 分析现有的 JSON 数据及其子问题。
2. 设计三个新的逻辑推理子问题。
3. 编写对应的参考答案。
4. 确保格式的一致性与准确性。

## JSON数据如下
{data}
"""
    return prompt


In [40]:
def read_file(ifn):
    # 读取输入文件
    data = []
    with open(ifn) as reader:
        for line in reader:
            # 解析每一行JSON数据
            sample = json.loads(line)
            data.append(sample)

    return data

In [41]:
data = read_file('data/round1_train_data.jsonl')

In [43]:
query = get_question_prompt(data[0])
call_qwen_api(MODEL_NAME, query)

'```json\n{\n  "problem": "有一个英文到法文的词汇表，包含以下对应词汇：\\n\\n1. the -> le\\n2. cat -> chat\\n3. jumps -> sauts\\n4. over -> sur\\n5. moon -> lune\\n6. cow -> vache\\n7. plays -> joue\\n8. fiddle -> violon\\n9. egg -> œuf\\n10. falls -> tombe\\n11. off -> de\\n12. wall -> mur\\n\\n根据这个词汇表，翻译以下英文句子成法文：",\n  "questions": [\n    {\n      "question": "选择题 1：\\n英文句子 \\"the cat jumps over the moon\\" 翻译成法文是：",\n      "options": ["le chat saute sur la lune", "le chat sauts sur le lune", "le sauts chat sur le lune", "le chat sauts sur le lune"],\n      "answer": "A"\n    },\n    {\n      "question": "选择题 2：\\n英文句子 \\"the cow plays the fiddle\\" 翻译成法文是：",\n      "options": ["le vache jouer le violon", "le jouer vache le violon", "le vache jouer la vièle", "la vache joue le violon"],\n      "answer": "D"\n    },\n    {\n      "question": "选择题 3：\\n英文句子 \\"the egg falls off the wall\\" 翻译成法文是：",\n      "options": ["le bougre des chutes de le mur", "le oeuf tombe de le mur", "le bougre tomber de le mur",